# Lab 1: Create Investment Agents Locally

This notebook demonstrates creating a 2-agent investment research system locally, following the 07-AgentCore-E2E pattern with agents inspired by runtime_ivr.

## Overview

In this lab, we'll build:
- **Quantitative Analysis Agent**: Stock data analysis and portfolio optimization (inspired by runtime_ivr)
- **Investment Research Assistant**: Supervisor agent that coordinates analysis and provides recommendations

This follows the **Lab 1** pattern from 07-AgentCore-E2E: create agents locally first, then add memory (Lab 2), then deploy to runtime (Lab 3).

## Prerequisites

- Python 3.10+
- AWS credentials configured
- Amazon Bedrock AgentCore access
- Required Python packages (see requirements.txt)

## Step 1: Environment Setup and Authentication

First, let's install the required packages and set up AWS authentication with temporary credentials.

In [ ]:
# Install required packages from requirements.txt
!pip install -q -r requirements.txt

In [ ]:
# Import required libraries
import os
import boto3
import json
import uuid
from datetime import datetime
from IPython.display import display, Markdown

# Strands imports
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import think

print("✅ All libraries imported successfully")

In [ ]:
# Set AWS credentials - Update these with your temporary credentials only for local machine
os.environ['AWS_ACCESS_KEY_ID'] = 'AWS_ACCESS_KEY_ID'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'AWS_SECRET_ACCESS_KEY'
os.environ['AWS_SESSION_TOKEN'] = 'AWS_SESSION_TOKEN'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

print("AWS credentials set successfully!")

# Clear boto3 cached sessions
boto3.DEFAULT_SESSION = None

# Verify credentials and extract account ID
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    AWS_ACCOUNT_ID = identity['Account']
    AWS_REGION = os.environ['AWS_DEFAULT_REGION']
    
    print(f"✅ Using AWS Account: {AWS_ACCOUNT_ID}")
    print(f"✅ User/Role: {identity['Arn']}")
    print(f"✅ Region: {AWS_REGION}")
except Exception as e:
    print(f"❌ Credential verification failed: {e}")
    raise

## Step 2: Create Quantitative Analysis Agent

This agent is inspired by the runtime_ivr notebook and provides:
- Stock data lookup with historical prices and metrics
- Portfolio optimization using Modern Portfolio Theory
- Technical analysis capabilities

We'll use `%%writefile` to create the agent as a separate Python file, following the runtime_ivr pattern.

In [ ]:
%%writefile lab_helpers/lab1_quantitative_agent.py
import datetime as dt
import numpy as np
import pandas as pd
import json
from typing import Dict, Union, List
import yfinance as yf
from scipy.optimize import minimize

from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import think

@tool
def stock_data_lookup(ticker: str, period: str = "1mo") -> Union[Dict, str]:
    """Gets historical stock price data for a given stock ticker."""
    try:
        if not ticker.strip():
            return {"status": "error", "message": "Ticker symbol is required"}

        stock = yf.Ticker(ticker)
        data = stock.history(period=period)
        
        if data.empty:
            return {"status": "error", "message": f"No data found for ticker {ticker}"}

        # Calculate key metrics
        current_price = float(data["Close"].iloc[-1])
        previous_close = float(data["Close"].iloc[-2]) if len(data) > 1 else current_price
        price_change = current_price - previous_close
        price_change_percent = (price_change / previous_close) * 100 if previous_close != 0 else 0
        
        # Calculate volatility
        returns = data["Close"].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100  # Annualized volatility
        
        return {
            "status": "success",
            "data": {
                "symbol": ticker,
                "current_price": round(current_price, 2),
                "previous_close": round(previous_close, 2),
                "price_change": round(price_change, 2),
                "price_change_percent": round(price_change_percent, 2),
                "volume": int(data["Volume"].iloc[-1]),
                "high": round(float(data["High"].max()), 2),
                "low": round(float(data["Low"].min()), 2),
                "volatility": round(volatility, 2),
                "data_points": len(data),
                "period": period,
                "date": dt.datetime.now().strftime("%Y-%m-%d")
            }
        }
    except Exception as e:
        return {"status": "error", "message": f"Error fetching price data: {str(e)}"}

@tool
def portfolio_optimization(tickers: List[str], investment_amount: float = 10000) -> Union[Dict, str]:
    """Optimizes portfolio allocation using Modern Portfolio Theory."""
    try:
        if not tickers or len(tickers) < 2:
            return {"status": "error", "message": "At least 2 tickers required for portfolio optimization"}
        
        # Get historical data for all tickers
        stock_data = {}
        for ticker in tickers:
            stock = yf.Ticker(ticker)
            data = stock.history(period="1y")
            if not data.empty:
                stock_data[ticker] = data["Close"]
        
        if len(stock_data) < 2:
            return {"status": "error", "message": "Insufficient data for portfolio optimization"}
        
        # Create returns dataframe
        returns_df = pd.DataFrame(stock_data).pct_change().dropna()
        
        # Calculate expected returns and covariance matrix
        expected_returns = returns_df.mean() * 252  # Annualized returns
        cov_matrix = returns_df.cov() * 252  # Annualized covariance
        
        # Portfolio optimization function
        def portfolio_stats(weights):
            portfolio_return = np.sum(expected_returns * weights)
            portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            sharpe_ratio = portfolio_return / portfolio_volatility if portfolio_volatility != 0 else 0
            return portfolio_return, portfolio_volatility, sharpe_ratio
        
        # Optimization constraints
        n_assets = len(tickers)
        constraints = (
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # weights sum to 1
        )
        bounds = tuple((0, 1) for asset in range(n_assets))  # weights between 0 and 1
        
        # Initial guess (equal weights)
        initial_weights = np.array([1/n_assets] * n_assets)
        
        # Optimize for maximum Sharpe ratio
        result = minimize(
            lambda x: -portfolio_stats(x)[2],  # Maximize Sharpe ratio
            initial_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        
        if result.success:
            optimal_weights = result.x
            portfolio_return, portfolio_volatility, sharpe_ratio = portfolio_stats(optimal_weights)
            
            # Calculate allocation amounts
            allocations = {}
            for i, ticker in enumerate(tickers):
                allocations[ticker] = {
                    "weight": round(optimal_weights[i] * 100, 2),
                    "amount": round(optimal_weights[i] * investment_amount, 2)
                }
            
            return {
                "status": "success",
                "data": {
                    "tickers": tickers,
                    "investment_amount": investment_amount,
                    "allocations": allocations,
                    "expected_return": round(portfolio_return * 100, 2),
                    "expected_volatility": round(portfolio_volatility * 100, 2),
                    "sharpe_ratio": round(sharpe_ratio, 3),
                    "optimization_method": "Maximum Sharpe Ratio",
                    "date": dt.datetime.now().strftime("%Y-%m-%d")
                }
            }
        else:
            return {"status": "error", "message": "Portfolio optimization failed"}
            
    except Exception as e:
        return {"status": "error", "message": f"Error in portfolio optimization: {str(e)}"}

def create_quantitative_analysis_agent():
    """Create and configure the quantitative analysis agent."""
    return Agent(
        name="quantitative_analysis_agent",
        system_prompt="""You are a Quantitative Analysis Agent specializing in financial data analysis and portfolio optimization.

Your responsibilities:
1. Query and analyze historical stock data using stock_data_lookup
2. Build optimized portfolio allocations using portfolio_optimization
3. Provide detailed analysis of stock performance, trends, and risk metrics
4. Calculate key financial ratios and statistical measures

When analyzing stock data:
- Focus on price movements, volume analysis, and volatility
- Identify trends and patterns in historical data
- Calculate risk-adjusted returns and performance metrics

When optimizing portfolios:
- Use Modern Portfolio Theory principles
- Consider risk-return tradeoffs
- Provide clear allocation recommendations with reasoning

Always provide structured, quantitative analysis with specific metrics and actionable insights.""",
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0", region_name="us-west-2"),
        tools=[stock_data_lookup, portfolio_optimization, think]
    )

print("✅ Quantitative Analysis Agent functions defined")

## Step 3: Test Quantitative Analysis Agent Locally

Let's test the QA agent locally to ensure it works correctly before proceeding.

In [ ]:
# Import the QA agent we just created
from lab_helpers.lab1_quantitative_agent import create_quantitative_analysis_agent

# Create the agent
qa_agent = create_quantitative_analysis_agent()

print("✅ Quantitative Analysis Agent created successfully")
print(f"Agent name: {qa_agent.name}")
print("✅ Agent configured with stock analysis and portfolio optimization tools")

In [ ]:
# Test the QA agent with a stock data query
print("🧪 Testing QA Agent with stock data lookup...\n")

test_query = "Get the current stock data for AAPL for the last month and analyze the key metrics"
response = qa_agent(test_query)

print("📊 QA Agent Response:")
display(Markdown(response.message["content"][0]["text"]))

In [ ]:
# Test the QA agent with portfolio optimization
print("🧪 Testing QA Agent with portfolio optimization...\n")

test_query = "Optimize a portfolio with AAPL, MSFT, and GOOGL for a $10,000 investment"
response = qa_agent(test_query)

print("📈 QA Agent Portfolio Optimization Response:")
display(Markdown(response.message["content"][0]["text"]))

## Step 4: Create Investment Research Assistant (Supervisor)

This supervisor agent coordinates the QA agent and provides comprehensive investment analysis. It will later be enhanced with memory in Lab 2.

In [ ]:
%%writefile lab_helpers/lab1_supervisor_agent.py
import json
from typing import Dict, Any, List
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import think

# Import the QA agent
from lab_helpers.lab1_quantitative_agent import create_quantitative_analysis_agent

@tool
def quantitative_analysis(query: str) -> str:
    """Delegate quantitative analysis tasks to the specialized QA agent."""
    try:
        qa_agent = create_quantitative_analysis_agent()
        response = qa_agent(query)
        return response.message["content"][0]["text"]
    except Exception as e:
        return f"Error in quantitative analysis: {str(e)}"

def create_investment_research_assistant():
    """Create and configure the investment research assistant supervisor agent."""
    return Agent(
        name="investment_research_assistant",
        system_prompt="""You are an Investment Research Assistant, a financial research supervisor coordinating specialized analysis.

Your responsibilities:
1. Break down complex investment research queries into subtasks
2. Delegate quantitative analysis tasks to the quantitative_analysis tool
3. Synthesize analysis results into comprehensive investment insights
4. Generate final investment research reports with clear recommendations

Workflow:
1. Analyze user query to identify required information
2. Use quantitative_analysis tool for stock data, portfolio optimization, and technical analysis
3. Synthesize information into structured investment analysis
4. Provide actionable investment recommendations

When coordinating analysis:
- Use quantitative_analysis for stock price data, historical analysis, and portfolio optimization
- Focus on risk-return analysis and portfolio construction
- Provide clear investment thesis and supporting data

Always provide comprehensive, well-structured investment analysis with clear recommendations and supporting quantitative data.""",
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0", region_name="us-west-2"),
        tools=[quantitative_analysis, think]
    )

print("✅ Investment Research Assistant functions defined")

## Step 5: Test Investment Research Assistant Locally

Let's test the supervisor agent to ensure it properly coordinates with the QA agent.

In [ ]:
# Import the supervisor agent we just created
from lab_helpers.lab1_supervisor_agent import create_investment_research_assistant

# Create the supervisor agent
supervisor_agent = create_investment_research_assistant()

print("✅ Investment Research Assistant created successfully")
print(f"Agent name: {supervisor_agent.name}")
print("✅ Supervisor configured with quantitative analysis delegation tool")

In [ ]:
# Test the supervisor agent with a comprehensive investment query
print("🧪 Testing Investment Research Assistant with comprehensive analysis...\n")

test_query = "Analyze TSLA stock performance and provide an investment recommendation. Include current metrics and portfolio considerations."
response = supervisor_agent(test_query)

print("📋 Investment Research Assistant Response:")
display(Markdown(response.message["content"][0]["text"]))

## Step 6: Test Multi-Agent Coordination

Let's test the full system with a complex query that requires coordination between agents.

In [ ]:
# Test multi-agent coordination with a complex investment scenario
print("🧪 Testing Multi-Agent Coordination...\n")

complex_query = """I'm considering investing in tech stocks. 
Please analyze AAPL, MSFT, and NVDA individually, then create an optimized portfolio allocation for $25,000. 
Provide a comprehensive investment recommendation with risk analysis."""

response = supervisor_agent(complex_query)

print("🎯 Comprehensive Multi-Agent Analysis:")
display(Markdown(response.message["content"][0]["text"]))

## Step 7: Validation and Summary

Let's validate that our agents are working correctly and summarize what we've accomplished.

In [ ]:
# Validation summary
print("📋 Lab 1 Validation Summary")
print("=" * 50)

# Check if agent files were created
import os
qa_file_exists = os.path.exists('lab_helpers/lab1_quantitative_agent.py')
supervisor_file_exists = os.path.exists('lab_helpers/lab1_supervisor_agent.py')
utils_file_exists = os.path.exists('lab_helpers/utils.py')

print(f"✅ Quantitative Analysis Agent file created: {qa_file_exists}")
print(f"✅ Investment Research Assistant file created: {supervisor_file_exists}")
print(f"✅ Utils file created: {utils_file_exists}")

# Check agent functionality
try:
    qa_test = qa_agent("Get AAPL current price")
    qa_working = True
except:
    qa_working = False

try:
    supervisor_test = supervisor_agent("Analyze AAPL")
    supervisor_working = True
except:
    supervisor_working = False

print(f"✅ QA Agent functioning: {qa_working}")
print(f"✅ Supervisor Agent functioning: {supervisor_working}")
print(f"✅ Multi-agent coordination: {qa_working and supervisor_working}")

print("\n🎉 Lab 1 Complete!")

## Conclusion

🎉 **Congratulations! You have successfully completed Lab 1: Create Investment Agents Locally**

### What You Accomplished:

1. **Created Quantitative Analysis Agent** with:
   - Stock data lookup tool with comprehensive metrics
   - Portfolio optimization using Modern Portfolio Theory
   - Technical analysis capabilities

2. **Created Investment Research Assistant** (Supervisor) with:
   - Agent coordination and task delegation
   - Investment analysis synthesis
   - Comprehensive recommendation generation

3. **Validated Multi-Agent System** with:
   - Local agent functionality testing
   - Agent coordination verification
   - Complex investment scenario handling

### Key Features Demonstrated:

- **Agent Architecture**: Following runtime_ivr patterns with robust tools
- **Tool Integration**: Stock data analysis and portfolio optimization
- **Agent Coordination**: Supervisor delegating to specialist agents
- **Local Development**: Testing and validation before deployment

### Files Created:
- `lab_helpers/lab1_quantitative_agent.py` - QA agent with financial tools
- `lab_helpers/lab1_supervisor_agent.py` - Supervisor agent
- `lab_helpers/utils.py` - Utility functions
- `requirements.txt` - Python dependencies

### Next Up: [Lab 2 - Add Memory →]

In Lab 2, we'll enhance the supervisor agent with AgentCore Memory to:
- Learn investor preferences across sessions
- Store comprehensive interaction history
- Provide personalized investment analysis
- Enable cross-session continuity

The foundation is now ready for intelligent, memory-enhanced investment analysis!